In [12]:
!pip install kaggle # Upload kaggle.json here

In [13]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"tanushakatakam","key":"4d681e49ee2e638d6a6cea0b418d3d8a"}'}

In [14]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [16]:
!kaggle datasets download sahilkirpekar/goodreads10k-dataset-cleaned

Dataset URL: https://www.kaggle.com/datasets/sahilkirpekar/goodreads10k-dataset-cleaned
License(s): CC0-1.0
  0% 0.00/6.84M [00:00<?, ?B/s]
100% 6.84M/6.84M [00:00<00:00, 885MB/s]


In [17]:
!unzip goodreads10k-dataset-cleaned.zip -d goodreads10k-dataset-cleaned

Archive:  goodreads10k-dataset-cleaned.zip
  inflating: goodreads10k-dataset-cleaned/Books.csv  
  inflating: goodreads10k-dataset-cleaned/Ratings.csv  
  inflating: goodreads10k-dataset-cleaned/to_read.csv  


In [24]:

import pandas as pd
import sqlite3

df = pd.read_csv("goodreads10k-dataset-cleaned/Books.csv")

mycon = sqlite3.connect("libdb.db")
cursor = mycon.cursor()


cursor.executescript("""
CREATE TABLE IF NOT EXISTS members(
    member_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    email TEXT,
    phone TEXT,
    is_deleted INTEGER DEFAULT 0
);

CREATE TABLE IF NOT EXISTS books(
    book_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    author TEXT,
    available INTEGER DEFAULT 1
);

CREATE TABLE IF NOT EXISTS borrow(
    borrow_id INTEGER PRIMARY KEY AUTOINCREMENT,
    member_id INTEGER,
    book_id INTEGER,
    issue_date TEXT,
    return_date TEXT,
    FOREIGN KEY (member_id) REFERENCES members(member_id),
    FOREIGN KEY (book_id) REFERENCES books(book_id)
);
""")

for _, row in df.iterrows():
    cursor.execute(
        "INSERT INTO books (Title, Author) VALUES (?, ?)",
        (row["Title"], row["Author"])
    )

mycon.commit()
mycon.close()

print("Database and tables created successfully.")


Database and tables created successfully.


In [29]:
import sqlite3
from datetime import datetime

mycon = sqlite3.connect("libdb.db")
cursor = mycon.cursor()

def add_member(name, email, phone):
    if name.isdigit():
      print("Invalid name, please enter name of string datatype")
      return
    cursor.execute("INSERT INTO members (name, email, phone) VALUES (?, ?, ?)", (name, email, phone))
    mycon.commit()
    print(f"Member added with ID : {cursor.lastrowid}")

def borrow_book(member_id, Title):
    cursor.execute("SELECT * FROM members WHERE member_id = ? AND is_deleted = 0", (member_id,))
    member = cursor.fetchone()
    if not member:
        print("Invalid or deleted member")
        return

    cursor.execute("SELECT book_id, available from Books WHERE title = ?", (Title,))
    book = cursor.fetchone()
    if not book:
      print("Book not found")
      return

    book_id, available = book
    if available:
        cursor.execute("INSERT INTO borrow (member_id, book_id, issue_date) VALUES (?, ?, ?)",(member_id, book_id, datetime.today().isoformat()))
        cursor.execute("UPDATE books SET available = 0 WHERE book_id = ?", (book_id,))
        mycon.commit()
        print("Book issued")
    else:
        print("Book not available")

def return_book(Title):
    cursor.execute("SELECT book_id FROM books WHERE title = ?", (Title,))
    book = cursor.fetchone()
    if not book:
        print("Book not found.")
        return
    book_id = book[0]
    cursor.execute("SELECT borrow_id FROM borrow WHERE book_id = ? AND return_date IS NULL", (book_id,))
    borrow = cursor.fetchone()
    if borrow:
        cursor.execute("UPDATE borrow SET return_date = ? WHERE borrow_id = ?", (datetime.today().isoformat(), borrow[0]))
        cursor.execute("UPDATE books SET available = 1 WHERE book_id = ?", (book_id,))
        mycon.commit()
        print("Book returned")
    else:
        print("No borrowed book")

def display_members():
    cursor.execute("SELECT member_id, name, email, phone FROM members WHERE is_deleted = 0")
    rows = cursor.fetchall()
    if rows:
        print("\nMembers List:")
        print("ID\tName\tEmail\tPhone")
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}")
    else:
        print("No members found.")

def display_deleted_members():
  cursor.execute("SELECT member_id, name, email, phone FROM members WHERE is_deleted = 1")
  rows = cursor.fetchall()
  if rows:
        print("\nMembers List:")
        print("ID\tName\tEmail\tPhone")
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}")
  else:
        print("No deleted members found.")

def delete_member(member_id):
  cursor.execute("UPDATE members SET is_deleted = 1 WHERE member_id = ?", (member_id,))
  mycon.commit()
  print(f"Member {member_id} is deleted")

def display_borrowed_books():
    cursor.execute("""SELECT b.borrow_id, m.name, bk.title, bk.author, b.issue_date
    FROM borrow b
    JOIN members m ON b.member_id = m.member_id
    JOIN books bk ON b.book_id = bk.book_id
    WHERE b.return_date IS NULL""")
    rows = cursor.fetchall()
    if rows:
        print("\nBorrowed Books:")
        print("BorrowID\tmember\tBook Title\tAuthor\tIssue Date")
        for row in rows:
            print(f"{row[0]}\t{row[1]}\t{row[2]}\t{row[3]}\t{row[4]}")
    else:
        print("No books found.")



In [31]:
while True:
    print("\nSELECT CHOICE")
    print("1. Add Member")
    print("2. Borrow Book")
    print("3. Return Book")
    print("4. Display Members")
    print("5. Display Borrowed Books")
    print("6. Delete Members")
    print("7. Display Deleted Members")
    print("8. Exit")

    choice = input("Enter choice : ")

    if choice == "1":
        name = input("Enter member name : ")
        email = input("Enter email : ")
        phone = input("Enter phone : ")
        add_member(name, email, phone)

    elif choice == "2":
        member_id = int(input("Enter member ID : "))
        title = input("Enter book Title : ")
        borrow_book(member_id, title)

    elif choice == "3":
        title = input("Enter book title to return : ")
        return_book(title)

    elif choice == "4":
        display_members()

    elif choice == "5":
        display_borrowed_books()

    elif choice == "6":
        member_id = int(input("Enter member ID to delete : "))
        delete_member(member_id)

    elif choice == "7":
        display_deleted_members()

    elif choice == "8":
        print ("Program Terminated")
        break

    else:
        print("Invalid choice")



SELECT CHOICE
1. Add Member
2. Borrow Book
3. Return Book
4. Display Members
5. Display Borrowed Books
6. Delete Members
7. Display Deleted Members
8. Exit
Enter choice : 6
Enter member ID to delete : 1
Member 1 is deleted

SELECT CHOICE
1. Add Member
2. Borrow Book
3. Return Book
4. Display Members
5. Display Borrowed Books
6. Delete Members
7. Display Deleted Members
8. Exit
Enter choice : 7

Members List:
ID	Name	Email	Phone
1	Alex	Alex12@gmail.com	325465765

SELECT CHOICE
1. Add Member
2. Borrow Book
3. Return Book
4. Display Members
5. Display Borrowed Books
6. Delete Members
7. Display Deleted Members
8. Exit
Enter choice : 4

Members List:
ID	Name	Email	Phone
2	Alex	Alex12@gmail.com	21328947
3	Alex	Alex12@gmail.com	435846554
4	Alex	Alex12@gmail.com	34354554
5	Alex	Alex12@gmail.com	352345621

SELECT CHOICE
1. Add Member
2. Borrow Book
3. Return Book
4. Display Members
5. Display Borrowed Books
6. Delete Members
7. Display Deleted Members
8. Exit
Enter choice : 6
Enter member ID 